https://radimrehurek.com/gensim/models/coherencemodel.html

In [13]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
model = LdaModel(common_corpus, 5, common_dictionary)

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
cm = CoherenceModel(model=model, corpus=common_corpus, coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
coherence

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-14.679371186797123

In [7]:
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel
topics = [
    ['human', 'computer', 'system', 'interface']
]

In [10]:
v

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [11]:
common_dictionary

In [8]:
cm = CoherenceModel(topics=topics, corpus=common_corpus, dictionary=common_dictionary, coherence='u_mass')
coherence = cm.get_coherence()  # get coherence value

In [9]:
coherence

-0.7607246985689725

In [4]:
# -*- coding: utf-8 -*- 
from konlpy.tag import Mecab 
from tqdm import tqdm 
import re 
import pickle 
import csv

ModuleNotFoundError: No module named 'konlpy'

In [2]:
def clean_text(text):
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text

In [3]:
def get_nouns(tokenizer, sentence): 
#     """ 단어의 길이가 2이상인 일반명사(NNG), 고유명사(NNP), 외국어(SL)만을 반환한다. 
#     :param tokenizer: :param sentence: :return: """ 
    tagged = tokenizer.pos(sentence) 
    nouns = [s for s, t in tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1]
    return nouns

In [ ]:
def tokenize(df):
    tokenizer = Mecab(dicpath='/Users/kimkwangil/Documents/001_ESSAYFITAI/Accu-Critique/CollegeSuppEssay') 
    processed_data = [] 
    for sent in tqdm(df['description']): 
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
    return processed_data


In [ ]:
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    with open('./data/tokenized_data.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data: 
            writer.writerow(data) 

if __name__ == '__main__': 
    # 유공자 공훈조서 데이터를 읽어들인다. 
    df = pd.read_csv('./historical_records.csv') 
    processed_data = tokenize(df) # description 부분을 토크나이징 한다. 
    # 토큰 분리한 데이터를 저장 
    save_processed_data(processed_data)

In [7]:
# -*- coding: utf-8 -*- 
from konlpy.tag import Mecab 
from tqdm import tqdm 
import re 
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric
import logging
import pickle
#import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [8]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step): 
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics) 
        model_list.append(model) 
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [9]:
def find_optimal_number_of_topics(dictionary, corpus, processed_data):
    limit = 40;
    start = 2;
    step = 6;
    
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step)
    
    x = range(start, limit, step)
    
    plt.plot(x, coherence_values)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [10]:
if __name__ == '__main__': 
    processed_data = [sent.strip().split(",") for sent in tqdm(open('./tokenized_data.csv', 'r', encoding='utf-8').readlines())] 
    # 정수 인코딩과 빈도수 생성 
    dictionary = corpora.Dictionary(processed_data) 
    # 출현빈도가 적거나 자주 등장하는 단어는 제거 
    dictionary.filter_extremes(no_below=10, no_above=0.05) 
    corpus = [dictionary.doc2bow(text) for text in processed_data] 
    print('Number of unique tokens: %d' % len(dictionary)) 
    print('Number of documents: %d' % len(corpus)) 
    
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 
    
    # 최적의 토픽 수 찾기 find_optimal_number_of_topics(dictionary, corpus, processed_data)

100%|██████████| 15925/15925 [00:00<00:00, 155745.57it/s]


Number of unique tokens: 5998
Number of documents: 15925


In [13]:
processed_data

[['일본',
  '동경',
  '박열',
  '흑도회',
  '기관지',
  '흑도',
  '흑우회',
  '기관지',
  '태선인',
  '현사회',
  '간행',
  '무정부주의',
  '선전',
  '회원',
  '규합',
  '노력',
  '박열',
  '주도',
  '무정부주의',
  '단체',
  '불령사',
  '가입',
  '활동',
  '동년',
  '동경',
  '대지진',
  '직후',
  '체포',
  '박열',
  '대역사건',
  '기소',
  '사형',
  '이후',
  '무기징역',
  '감형',
  '우쓰노미야형무소',
  '우도궁형무소',
  '도치기지소',
  '회목지소',
  '수감',
  '옥중',
  '순국',
  '박열',
  '의기투합',
  '일본',
  '제국주의',
  '반대',
  '활동',
  '시작',
  '흑도회',
  '가입',
  '박열',
  '기관지',
  '흑도',
  '발간',
  '책임',
  '흑우회',
  '조직',
  '사상연구',
  '연설회',
  '추진',
  '기관지',
  '민중운동',
  '한글',
  '발행',
  '후토이센진',
  '창간',
  '현사회',
  '이름',
  '개칭',
  '지속',
  '발간',
  '무정부주의',
  '운동가',
  '규합',
  '불령사',
  '조직',
  '불령사',
  '한국',
  '형평운동',
  '지지',
  '한편',
  '노동운동',
  '적극',
  '지원',
  '무정부주의',
  '입각',
  '항일투쟁',
  '전개',
  '의열단',
  '의열투쟁',
  '추진',
  '폭탄',
  '도쿄',
  '반입',
  '일왕',
  '정부',
  '요인',
  '암살',
  '계획',
  '도쿄대지진',
  '직후',
  '보호',
  '감속',
  '명목',
  '도쿄',
  '세타가야',
  '경찰서',
  '구금',
  '취조',
  '도중',
  '폭탄',
  '반입',
  '사실',
  '대역사건',
  '주

In [11]:
dictionary

In [12]:
corpus

[[(0, 1),
  (1, 2),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 2),
  (6, 4),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 2),
  (15, 1),
  (16, 2),
  (17, 4),
  (18, 1),
  (19, 2),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 2),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 2),
  (48, 1),
  (49, 2),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2)],
 [(18, 2),
  (55, 1),
  (56, 1),
  (57, 2),
  (58, 1),
  (59, 1),
  (60, 2),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 2),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 2),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 3

In [53]:
import re
import nltk
from nltk import sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from gensim import corpora

In [54]:
# input College Supp Essay 
essay_input = """I inhale deeply and blow harder than I thought possible, pushing the tiny ember from its resting place on the candle out into the air. The room erupts around me, and 'Happy Birthday!' cheers echo through the halls. It's time to make a wish. In my mind, that new Limited Edition Deluxe Ben 10 watch will soon be mine. My parents and the aunties and uncles around me attempt to point me in a different direction. 'Wish that you get to go to the temple every day when you're older! Wish that you memorize all your Sanskrit texts before you turn 6! Wish that you can live in India after college!' My ears listen, but my mind tunes them out, as nothing could possibly compare to that toy watch! What I never realized on my third birthday is that those wishes quietly tell the story of how my family hopes my life will play out. In this version of my life, there wasn't much room for change, personal growth, or 'rocking the boat.' A vital aspect of my family's cultural background is their focus on accepting things as they are. Growing up, I was discouraged from questioning others or asking questions that didn't have definitive yes or no answers. If I innocently asked my grandma why she expected me to touch her feet, my dad would grab my hand in a sudden swoop, look me sternly in the eye, and tell me not to disrespect her like that again. At home, if I mentioned that I had tried eggs for breakfast at a friend's house, I'd be looked at like I had just committed a felony for eating what my parents considered meat. If I asked the priest at the temple why he had asked an Indian man and his white wife to leave, I'd be met with a condescending glare and told that I should also leave for asking such questions.In direct contrast, my curiosity was invited and encouraged at school. After an environmental science lesson, I stayed for a few minutes after class to ask my 4th-grade science teacher with wide eyes how it was possible that Niagara Falls doesn't run out of flowing water. Instead of scolding me for asking her a 'dumb question,' she smiled and explained the intricacy of the water cycle. Now, if a teacher mentions that we'll learn about why a certain proof or idea works only in a future class, I'll stay after to ask more or pour through an advanced textbook to try to understand it. While my perspective was widening at school, the receptiveness to raising complex questions at home was diminishing. After earning my driver's license, I registered as an organ donor. My small checkmark on a piece of paper led to an intense clash between my and my parents' moral platform. I wanted to ensure that I positively contributed to society, while my parents believed that organ donation was an unfamiliar and unnecessary cultural taboo. I would often ask for clarity or for reasons that supported their ideologies. Their response would usually entail feeling a deep, visceral sense that traditions must be followed exactly as taught, without objection. Told in one language to keep asking questions and in another to ask only the right ones, I chose exploring questions that don't have answers, rather than accepting answers that don't get questioned. When it comes to the maze of learning, even when I take a wrong turn and encounter roadblocks that are meant to stop me, I've learned to climb over them and keep moving forward. My curiosity strengthens with each hurdle and has expanded into a pure love of learning new things. I've become someone who seeks to understand things at a fundamental level and who finds excitement in taking on big questions that have yet to be solved. I'm no longer afraid to rock the boat. "},{"index":1,"personal_essay":"Ever since I first held a small foam Spiderman basketball in my tiny hands and watched my idol Kobe Bryant hit every three-pointer he attempted, I've wanted to understand and replicate his flawless jump shot. As my math education progressed in school, I began to realize I had the tools to create a perfect shot formula. After learning about variables for the first time in 5th grade Algebra, I began to treat each aspect of Kobe's jump shot as a different variable, each combination of variables resulting in a unique solution. While in 7th-grade geometry, I graphed the arc of his shot, and after learning about quadratic equations in 8th grade, I expressed his shot as a parabolic function that would ensure a swish when shooting from any spot. After calculus lessons in 10th and 11th grade, I was excited to finally solve for the perfect velocity and acceleration needed on my release. At Brown, I hope to explore this intellectual pursuit through a different lens. What if I could maximize the odds of making shots if I understood the science behind one's mental mindset and focus through CLPS 500: Perception and Action? Or use astrophysics to account for drag and gravitational force anywhere in the universe? Or use data science to break down the analytics of the NBA's best shooters? Through the Open Curriculum, I see myself not only becoming a more complete learner, but also a more complete thinker, applying a flexible mindset to any problem I encounter. Brown's Open Curriculum allows students to explore broadly while also diving deeply into their academic pursuits. Tell us about an academic interest (or interests) that excites you, and how you might use the Open Curriculum to pursue it. I've been playing the Mridangam since I was five years old. It's a simple instrument: A wood barrel covered on two ends by goatskin with leather straps surrounding the hull. This instrument serves as a connection between me and one of the most beautiful aspects of my culture: Carnatic music. As a young child, I'd be taken to the temple every weekend for three-hour-long Carnatic music concerts, where the most accomplished teenagers and young adults in our local Indian community would perform. I would watch in awe as the mridangists' hands moved gracefully, flowing across the goatskin as if they weren't making contact, while simultaneously producing sharp rhythmic patterns that never failed to fall on the beat. Hoping to be like these idols on the stage, I trained intensely with my teacher, a strict man who taught me that the simple drum I was playing had thousands of years of culture behind it. Building up from simple strokes, I realized that the finger speed I'd had been awestruck by wasn't some magical talent, it was instead a science perfected by repeated practice."""


In [55]:
# tokenize sentence
sent_token_re = sent_tokenize(essay_input)

In [56]:
processed_data = []
for i in sent_token_re:
    tokens_re = nltk.word_tokenize(i)
    # stopwords 제거, 각 문장을 소문자로 변환하고, 명사만 추출, 문장별로 리스트 만들어서 리스트에 다시 담기
    stop = set(stopwords.words('english'))
    clean_tokens = [tok for tok in tokens_re if len(tok.lower())>1 and (tok.lower() not in stop)]
    tagged = nltk.pos_tag(clean_tokens)
    allnoun = [word for word, pos in tagged if pos in ['NN','NNP']]
    processed_data.append(allnoun)

In [57]:
processed_data

[['harder', 'ember', 'place', 'candle', 'air'],
 ['room', 'Birthday'],
 [],
 ['time'],
 ['Limited', 'Edition', 'Deluxe', 'Ben', 'watch', 'mine'],
 ['point', 'direction'],
 ['get', 'day'],
 ['memorize', 'Sanskrit', 'texts', 'turn'],
 ['India', 'college'],
 ['nothing', 'toy', 'watch'],
 ['birthday', 'story', 'family', 'life', 'play'],
 ['version', 'life', 'room', 'change', 'growth', 'boat'],
 ['family', 'background', 'focus'],
 [],
 ['grandma', 'hand', 'swoop', 'look', 'eye', 'disrespect'],
 ['home', 'friend', 'house', 'felony', 'meat'],
 ['temple',
  'man',
  'wife',
  'glare',
  'questions.In',
  'contrast',
  'curiosity',
  'school'],
 ['science',
  'lesson',
  'class',
  'science',
  'teacher',
  'Niagara',
  'Falls',
  'water'],
 ['question', 'water', 'cycle'],
 ['teacher', 'proof', 'idea', 'class', 'pour', 'try', 'understand'],
 ['perspective', 'school', 'receptiveness', 'home', 'diminishing'],
 ['driver', 'license', 'donor'],
 ['checkmark', 'piece', 'paper', 'clash', 'platform'],


In [60]:
dictionary = corpora.Dictionary(processed_data)
dictionary

In [11]:
def get_nouns(sentence): 
    tagged = nltk.pos_tag(sentence) 
    nouns = [s for s, t in tagged if t in ['SL', 'NNG', 'NNP'] and len(s) > 1] 
    return nouns

In [12]:
get_nouns(essay_input)

[]

In [3]:
def clean_text(text):
    #영문, 숫자만 남기고 제거한다. :param text: :return: 
    pattern = '[^|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl=' ', string=text) 
    return text

In [4]:
cln_re = clean_text(essay_input)
cln_re

'I inhale deeply and blow harder than I thought possible pushing the tiny ember from its resting place on the candle out into the air The room erupts around me and Happy Birthday cheers echo through the halls It s time to make a wish In my mind that new Limited Edition Deluxe Ben 10 watch will soon be mine My parents and the aunties and uncles around me attempt to point me in a different direction Wish that you get to go to the temple every day when you re older Wish that you memorize all your Sanskrit texts before you turn 6 Wish that you can live in India after college My ears listen but my mind tunes them out as nothing could possibly compare to that toy watch What I never realized on my third birthday is that those wishes quietly tell the story of how my family hopes my life will play out In this version of my life there wasn t much room for change personal growth or rocking the boat A vital aspect of my family s cultural background is their focus on accepting things as they are Gr

In [24]:


get_nouns(cln_re)

[]

In [25]:
def tokenize(txt): 
    sent_token =  sent_tokenize(txt)
    result_dt = []
    for snt in sent_token:
        tokenizer = word_tokenize
        ct_re = clean_text(tokenizer)
        get_nouns_re = get_nouns(ct_re)
        wd_token_re = tokenizer(get_nouns_re)
        result_dt.append(wd_token_re)

    return result_dt